In [1]:
from keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
#performing data agumentation on test data
test_datagen = ImageDataGenerator(rescale = 1./255)

In [3]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [7]:
training_set = train_datagen.flow_from_directory('drive/MyDrive/im/train_set',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 316 images belonging to 3 classes.


In [8]:
test_set = test_datagen.flow_from_directory('drive/MyDrive/im/test_set',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 120 images belonging to 3 classes.


In [9]:
IMAGE_SIZE = [224, 224]

train_path ="drive/MyDrive/im/train_set"
valid_path ='drive/MyDrive/im/test_set'

In [10]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


58892288/58889256 [==============================] - 0s 0us/step


In [11]:
for layer in vgg16.layers:
    layer.trainable = False
    

In [12]:
folders = glob('drive/MyDrive/im/train_set/*')

In [13]:
folders


['drive/MyDrive/im/train_set/flakes',
 'drive/MyDrive/im/train_set/roof',
 'drive/MyDrive/im/train_set/crack']

In [14]:
x = Flatten()(vgg16.output)

In [15]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [17]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [18]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
10/10 [==============================] - 284s 28s/step - loss: 1.3704 - accuracy: 0.4810 - val_loss: 1.1813 - val_accuracy: 0.5083
Epoch 2/10
10/10 [==============================] - 220s 23s/step - loss: 0.6501 - accuracy: 0.7310 - val_loss: 1.3002 - val_accuracy: 0.5750
Epoch 3/10
10/10 [==============================] - 241s 25s/step - loss: 0.5354 - accuracy: 0.7690 - val_loss: 0.6464 - val_accuracy: 0.7250
Epoch 4/10
10/10 [==============================] - 241s 25s/step - loss: 0.3561 - accuracy: 0.8576 - val_loss: 0.5603 - val_accuracy: 0.7833
Epoch 5/10
10/10 [==============================] - 219s 23s/step - loss: 0.2257 - accuracy: 0.9209 - val_loss: 0.6490 - val_accuracy: 0.7667
Epoch 6/10
10/10 [==============================] - 219s 23s/step - loss: 0.1736 - accuracy: 0.9652 - val_loss: 0.5855 - val_accuracy: 0.7583
Epoch 7/10
10/10 [==============================] - 220s 23s/step - loss: 0.1672 - accuracy: 0.9557 - val_loss: 0.6442 - val_accuracy: 0.7750
Epoch 

In [19]:

from tensorflow.keras.models import load_model

model.save('model_building_defects_vgg16.h5')

In [20]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("model_building_defects_vgg16.h5")

In [21]:
img=image.load_img(r'drive/MyDrive/im/test_set/crack/1.jpg',target_size=(224,224))
x=image.img_to_array(img)
#x=x/255
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
#model.predict(img_data)
a=np.argmax(model.predict(img_data), axis=1)

In [22]:
a

array([0])

In [23]:
index=['crack','flakes','roof']
result=str(index[a[0]])
result

'crack'